# Processing the final dataset



In [1]:
import ee
import geemap

# Authenticate to Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

data_folder = "projects/amazon-forest-regrowth/assets"


In [17]:
amaz = ee.FeatureCollection(f"{data_folder}/raw/biomes_br") \
    .filter(ee.Filter.eq("CD_Bioma", 1)).geometry()  # 1 = Amazonia

all_country = ee.FeatureCollection(f"{data_folder}/raw/biomes_br").geometry().dissolve()


In [28]:
def export_clipped_image(image, scale, region, export_type, image_name=None):
    """
    Clips and exports image to Google Earth Engine asset and/or Google Drive.

    Parameters:
    - image: ee.Image, the image to export.
    - scale: int, the scale in meters.
    - region: str, 'amaz' or 'all_country'.
    - export_type: str, 'asset', 'drive', or 'both'.
    """

    export_region = amaz if region == 'amaz' else all_country
    
    # Get the image name for exports
    # Attempt to retrieve the image name from system:id
    if image_name is None:
        try:
            image_name = image.get('system:id').getInfo().split('/')[-1]
        except:
            raise ValueError("Image name not found in system:id. Please define image_name parameter.")
    
    # Prepare export parameters
    export_params = {
        'image': image.clip(export_region),
        'scale': scale,
        'region': export_region,
        'crs': "EPSG:4326",
        'maxPixels': 1e13
    }
    
    # Set additional parameters for each export type
    if export_type in ['asset', 'both']:
        asset_params = export_params.copy()
        asset_params['assetId'] = f"projects/amazon-forest-regrowth/assets/{image_name}_{scale}m_{region}"
        asset_params['description'] = f"{image_name}_{scale}m_{region}_toAsset"
        asset_task = ee.batch.Export.image.toAsset(**asset_params)
        asset_task.start()
        print(f"Asset export task for {image_name} in {region} started.")
    
    if export_type in ['drive', 'both']:
        drive_params = export_params.copy()
        drive_params['fileNamePrefix'] = f'{image_name}_{scale}m_{region}'
        drive_params['description'] = f"{image_name}_{scale}m_{region}_toDrive"
        drive_task = ee.batch.Export.image.toDrive(**drive_params)
        drive_task.start()
        print(f"Drive export task for {image_name} in {region} started.")



In [24]:
mature_biomass = ee.Image(f"{data_folder}/mature_biomass")

# Export to Drive for all country
export_clipped_image(mature_biomass, 500, 'all_country', 'both')


Asset export task for mature_biomass in all_country started.
Drive export task for mature_biomass in all_country started.


In [29]:
mature_biomass_aggregated = ee.Image(f"{data_folder}/mature_biomass_500m")

age_agbd = ee.Image(f"{data_folder}/age_agbd")

distance = mature_biomass_aggregated.fastDistanceTransform().sqrt() \
    .multiply(ee.Image.pixelArea().sqrt()).rename("distance") \
    .updateMask(age_agbd.select("age")).float()

# Export to Drive for all country
export_clipped_image(distance, 1000, 'all_country', 'both', 'distance')

Asset export task for distance in all_country started.
Drive export task for distance in all_country started.


In [12]:
distance = ee.Image(f"{data_folder}/distance_1000m")
distance_amaz = ee.Image(f"{data_folder}/distance_500m_amaz")

# map = geemap.Map()
# map.addLayer(distance, {"min": 0, "max": 180000, "palette": ["red", "blue"]}, "distance")
# map.addLayer(mature_biomass_aggregated, {"min": 0, "max": 300, "palette": ["yellow", "green"]}, "mature_biomass_aggregated")
# # map.addLayer(age_agbd.select("age"), {"min": 0, "max": 35}, "age")
# map.addLayer(mature_biomass, {"min": 0, "max": 300, "palette": ["yellow", "green"]}, "mature_biomass")
# map.addLayer(distance_amaz, {"min": 0, "max": 180000, "palette": ["red", "blue"]}, "distance_amaz")

# map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [27]:
def export_csv(year):
    if (year == 0):
        land_use_years = ee.Image(f"{data_folder}/land_use")
        name = "all_LULC_1000_countrywide"
    else:
        land_use_years = ee.Image(f"{data_folder}/land_use_{year}_years")
        name = f"{year}y_LULC_1000_countrywide"

    age_agbd = ee.Image(f"{data_folder}/age_agbd")
    categorical = ee.Image(f"{data_folder}/categorical")
    sur_cover = ee.Image(f"{data_folder}/sur_cover").rename("sur_cover")
    one_hectare_mask = ee.Image(f"{data_folder}/one_hectare_mask").rename("one_hectare_mask")
    cwd = ee.Image(f"{data_folder}/raw/cwd_chave").float().rename("cwd")
    yearly_mean_prec = ee.Image(f"{data_folder}/yearly_mean_prec")
    yearly_SI = ee.Image(f"{data_folder}/yearly_SI")
    mat = ee.Image(f"{data_folder}/nearest_mat_biomass_1000_countrywide").rename("distance", "mature_biomass")

    combined_mask = one_hectare_mask.And(land_use_years.select("last_LU")) \
        .And(mat.select("distance"))

    unified_data = (
        age_agbd.addBands(
            [mat, distance, categorical, land_use_years, cwd, sur_cover, yearly_mean_prec, yearly_SI
        ]).updateMask(combined_mask)
    )


    unified_data_fc = unified_data.stratifiedSample(
        numPoints = 5000, classBand = 'ecoreg', region = unified_data.geometry()
    )


    # Export an ee.FeatureCollection to Google Drive, adjusting the description based on the years
    task = ee.batch.Export.table.toDrive(
        collection=unified_data_fc, description=name
    )
    task.start()

export_csv(0)
export_csv(5)
export_csv(10)
export_csv(15)